# Docker Swarm
    Jens Egholm Pedersen - jensegholm@protonmail.com

## Agenda

* Introduction
* Docker
* Orchestration and Docker Swarm
* Recap

# Presentation formalia

* Who am I
* Who are you
* Presentation format

## Who I am

    jensegholm@protonmail.com

### Education

* BSc. Political Science
* BSc. IT-University of Copenhagen
* MSc. IT & Cognition

### Professional experience

* Software engineer, CERN, Switzerland
* Assistant professor, Cphbusiness
* CTO Mobilized Construction

# Who you are


* Probably similar to me

* Probably busy

<< Moore's law --> Law of accelerating returns >>

## Metacognition

* Think about how you think

* Science: You learn better when you write down

  * On paper

  * With a pen

# Presentation format

* Intuition
* Relevant
* Practical

* Theory
  * Listen, take notes, relate, ask
* Practice
  * Learn, internalise, criticise

# Docker

* Docker machine
* Containers
* Software synchronisation
* Synchronisation in containers

## Docker machine

* A tool to manage virtual machines
  * Locally or remotely
  * Drivers for many providers 

## Docker containers

* Simplification and isolition
  * Old, but good, idea

* Typical use case: microservices

* Is _not_ perfect for everything
  * Persistence (databases)
  * Synchronisation

## Software synchronisation

* Peripherals

* Filesystem

* Network
  * TCP, UDP
  * HTTP(S)
  * Websockets

## Synchronisation in containers

* No peripherals

* No filesystem

* Network is our only option
  * And it is comparatively slow

## Occam's razor

* Docker is not inviting complexity
* Docker is a framework for isolated containers
* It forces us to deal with complexity elsewhere

# Orchestration

# Recap